<a href="https://colab.research.google.com/github/batiukmaks/NLP-Labs/blob/main/labs/LPNLP_Workbook_3_Word_embeddings_for_classification_(2024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workbook 03: Word embeddings for text classification

У цій роботі ми використаємо word embeddings для тренування моделі класифікації текстів.

Маємо побачити, як word embeddings особливо сильно допомогають, коли тренувальних даних небагато (а їх майже завжди небагато).

In [1]:
!pip install --quiet --ignore-installed http://nlp.band/static/pypy/lpnlp-2023.10.2-py3-none-any.whl

In [2]:
import lpnlp
lab = lpnlp.start(
    email="maksym.batiuk.kn.2021@lpnu.ua",  # <---------------------- Заповніть це поле
    lab="using_word_embeddings",
    )


Удачі!


# GloVe

In [3]:
!pip install gensim

Повний GloVe містить 4,000,000 векторів і займає багато пам'яті. Щоб уникнути проблем з пам'ятю, залишимо лише 50,000 векторів найчастотніших слів. Це трохи знизить якість моделей, але це зараз не головне.

In [4]:
from gensim.models import KeyedVectors
glove = KeyedVectors.load("http://nlp.band/static/files/glove-50k.bin")

# Bag-of-embeddings

Вектори слів, натреновані на великому корпусі текстів, використовують для представлення слів замість розріджених one-hot, які ми бачили в першій лабораторній. Word embeddings чудово працюють з нейронним мережами різноманітних архітектур. Але зараз ми розглянемо напростіше використання: логістичну регресію (так, знову) та мішок векторів (bag-of-embeddings).

В bag-of-embeddings ми усереднюємо вектори всіх слів, які входять в речення. Результат — вектор такої ж розмірності, як і вектор слова. Цей вектор кодує зміст усього речення. Звичайно, таке представлення не враховує порядок слів, рівноцінно ставиться до важливих та допоміжних слів, а тому "кодує" воно зміст речення вельми приблизно. Проте цього достатньо для багатьох простих задач.

## Токенізація

Для початку нам треба токенізувати корпус. Важливий момент: GloVe та інші word embeddings тренувалися кожен зі своїм токенізатором. Нам слід використовувати максимально схожий токенізатор. Інакше ми не зможемо знайти вектори для багатьох слів.

Насамперед, GloVe тренувалися на тексті, приведеному до нижнього регістру. Також розбіжності можуть бути в кодуванні слів типу `I'll` (токенізується в два токени `I 'll` чи в один токен `I'll`?), `don't`, `I've` і подібних.

Перевіримо, який варіант токенізації використовує GloVe:

In [5]:
"don't" in glove

False

In [6]:
"n't" in glove

True

In [7]:
"I'll" in glove

False

In [8]:
"'ll" in glove

True

Отже, маємо розбивати `don't` на два токени: `do` + `n't`

In [9]:
import spacy
from typing import List


spacy_nlp = spacy.blank("en")


def tokenize(text: str) -> List[str]:
  """Tokenize string with SpaCy. """

  tokens = spacy_nlp.tokenizer(text)
  return [str(token).lower() for token in tokens]

tokenize("I don't know")

['i', 'do', "n't", 'know']

## Векторизація одного документа

Тепер можемо порахувати вектор для кожного документу в корпус. Цей вектор буде дорівнювати середньому від векторів окремих слів документа.

In [39]:
import numpy as np
from typing import Tuple

def bag_of_embeddings(doc: str, embeddings: KeyedVectors) -> np.ndarray:
    tokens = tokenize(doc)

    ##################################################
    doc_vector = np.average(
        np.array([
            embeddings[t]
            for t in tokens
            if t in embeddings
        ]),
        axis=0
    )
    ##################################################

    return doc_vector


doc_embedding = bag_of_embeddings("Hello world!", glove)
print(f"Embedding: {doc_embedding}")
print(f"Shape:     {doc_embedding.shape}")

Embedding: [ 0.41084     0.4957     -0.35982665 -0.40393332 -0.19768667  0.25433
 -0.51489997  0.086394    0.04418867  0.365309   -0.3562367   0.15675335
  0.09240001  0.4017      0.00335334 -0.08881667 -0.37311664  0.5704167
  0.04311933  0.34267667  0.47882667  1.489306    0.25857666 -0.19864707
  0.10432333  0.125766    0.10343501 -0.28578332 -0.31660533  0.013828
 -0.07466667  0.13855    -0.32214698 -0.28048036 -0.41403     0.06308667
 -0.239556   -0.03680335 -0.141137   -0.0916     -0.16916066  0.5716353
 -0.3125367   0.07347333 -0.16953166  0.20232     0.60658664  0.06445
 -0.01957001  0.30074     0.21663667 -0.06867133  0.4030467   0.07233366
 -0.17247333  0.20601167 -0.39144    -0.02245933 -0.17295867 -0.19998033
  0.234197   -0.29320666 -0.13344    -0.1644     -0.00662334  0.01097
 -0.00953534  0.5020967   0.42084002  0.03766001  0.46454334 -0.44661132
  0.22226368  0.0327     -0.43126     0.00774    -0.15634833  0.3885993
 -0.38001335 -0.13208102  0.22859664 -0.39621338  0.24

Розмірність вектора документа не залежить від кількості слів у ньому:

In [40]:
tests = [
    "Hello world",
    "You can try the best you can. The best you can is good enough.",
]
print("Розмір    Документ")
print("-" * 80)
for s in tests:
    shape = bag_of_embeddings("Hello world!", glove).shape
    print(f"{shape}    {s}")

Розмір    Документ
--------------------------------------------------------------------------------
(200,)    Hello world
(200,)    You can try the best you can. The best you can is good enough.


In [41]:
lab.checkpoint("`Hello world` centroid", bag_of_embeddings("Hello world!", glove).mean())

'0.0046237493'

In [42]:
glove['hello'].shape

(200,)

# Векторизація всього корпусу

Наступна операція може зайняти пару хвилин:

In [43]:
!pip install --quiet datasets

In [44]:
import datasets
imdb = datasets.load_dataset("imdb")

valid_data = imdb["test"].shuffle(seed=1).filter(lambda x, i: i < 2000, with_indices=True)  # take 2000 random rows for validation
train_data = imdb["train"].shuffle(seed=2)

In [45]:
from tqdm import tqdm


def vectorize_dataset(dataset: datasets.Dataset) -> Tuple[np.ndarray, np.ndarray]:
    """Векторізує весь датасет у представлення bag-of-embeddings.

    Повертає матрицю ознак X та вектор класів y.
    """
    X = []
    for doc in tqdm(dataset):
        doc_vector = bag_of_embeddings(doc["text"], glove)
        X.append(doc_vector)

    X = np.stack(X)
    y = np.array(dataset["label"])
    return (X, y)

In [46]:
X_train_boe, y_train_boe = vectorize_dataset(train_data)
X_valid_boe, y_valid_boe = vectorize_dataset(valid_data)
lab.checkpoint("Vectorized dataset shape", X_train_boe.shape)


100%|██████████| 2000/2000 [00:03<00:00, 509.77it/s]


(25000, 200)

## Logistic regression + Bag-of-Embeddings


In [47]:
# Штучно обмежимо кількість тренувальних прикладів цим значенням.
# Так ми емулюємо ситуацію, коли в нас мало тренувальних даних.
TRAIN_SIZE = 500

In [48]:
from sklearn.linear_model import LogisticRegression

# Тренуємо логістичну регресію
logreg = LogisticRegression(solver="liblinear")

logreg.fit(X_train_boe[:TRAIN_SIZE,], y_train_boe[:TRAIN_SIZE,])
logreg_acc = logreg.score(X_valid_boe, y_valid_boe)
lab.checkpoint(f"LogReg + BoE accuracy on {TRAIN_SIZE}", logreg_acc)

0.7325

## Logistic regression + TF-IDF

Натренуємо для порівняння модель на TF-IDF bag-of-ngrams ознаках. Тренувальні дані точнісінько такі, як і у моделі bag-of-embeddings. Але як щодо якості?


In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_bow = vectorizer.fit_transform(train_data[:TRAIN_SIZE]["text"])

model_tfidf = LogisticRegression(solver='liblinear', C=0.2, penalty="l1")
model_tfidf.fit(X_train_bow, train_data["label"][:TRAIN_SIZE])


X_valid_bow = vectorizer.transform(valid_data["text"])
y_valid_bow = valid_data["label"]
tfidf_acc = model_tfidf.score(X_valid_bow, y_valid_bow)
lab.checkpoint(f"LogReg + TF-IDF accuracy on {TRAIN_SIZE}", tfidf_acc)

0.5035

# Завдання

Перетренуйте моделі на різних розмірах `TRAIN_SIZE`. Спробуйте кілька значень. Зверніть увагу, як різницю між моделями змінюється в залежності від `TRAIN_SIZE`.

❗ Результат (посилання на ваш Google Colab або PDF) відправте на пошту oleksii.o.syvokon@lpnu.ua ❗

In [51]:
def test_logreg_bag_of_embeddings(train_size):
    logreg = LogisticRegression(solver="liblinear")

    logreg.fit(X_train_boe[:train_size,], y_train_boe[:train_size,])
    logreg_acc = logreg.score(X_valid_boe, y_valid_boe)
    return logreg_acc

def test_logreg_tf_idf(train_size):
    vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
    X_train_bow = vectorizer.fit_transform(train_data[:train_size]["text"])

    model_tfidf = LogisticRegression(solver='liblinear', C=0.2, penalty="l1")
    model_tfidf.fit(X_train_bow, train_data["label"][:train_size])


    X_valid_bow = vectorizer.transform(valid_data["text"])
    y_valid_bow = valid_data["label"]
    tfidf_acc = model_tfidf.score(X_valid_bow, y_valid_bow)
    return tfidf_acc

results = []
for train_size in [100, 200, 500, 1000, 5000, 10000, 20000, 25000]:
    logreg_acc = test_logreg_bag_of_embeddings(train_size)
    tfidf_acc = test_logreg_tf_idf(train_size)
    results.append((train_size, logreg_acc, tfidf_acc))

    print(f"LogReg + BoE accuracy on {train_size}", logreg_acc)
    print(f"LogReg + TF-IDF accuracy on {train_size}", tfidf_acc)
    print()

LogReg + BoE accuracy on 100 0.6285
LogReg + TF-IDF accuracy on 100 0.5035

LogReg + BoE accuracy on 200 0.6975
LogReg + TF-IDF accuracy on 200 0.5035

LogReg + BoE accuracy on 500 0.7325
LogReg + TF-IDF accuracy on 500 0.5035

LogReg + BoE accuracy on 1000 0.773
LogReg + TF-IDF accuracy on 1000 0.5035

LogReg + BoE accuracy on 5000 0.803
LogReg + TF-IDF accuracy on 5000 0.7415

LogReg + BoE accuracy on 10000 0.805
LogReg + TF-IDF accuracy on 10000 0.8

LogReg + BoE accuracy on 20000 0.8135
LogReg + TF-IDF accuracy on 20000 0.8295

LogReg + BoE accuracy on 25000 0.8135
LogReg + TF-IDF accuracy on 25000 0.8365



In [53]:
import pandas as pd

diff_acc_df = pd.DataFrame(results, columns=['train_size', 'logreg_acc', 'tfidf_acc'])
diff_acc_df['diff'] = diff_acc_df['logreg_acc'] - diff_acc_df['tfidf_acc']
diff_acc_df

,train_size,logreg_acc,tfidf_acc,diff
0,100,0.6285,0.5035,0.1250
1,200,0.6975,0.5035,0.1940
2,500,0.7325,0.5035,0.2290
3,1000,0.7730,0.5035,0.2695
4,5000,0.8030,0.7415,0.0615
5,10000,0.8050,0.8000,0.0050
6,20000,0.8135,0.8295,-0.0160
7,25000,0.8135,0.8365,-0.0230


**Аналіз результатів**: З результатів бачимо, що при значенні `TRAIN_SIZE` від 100 до 1000 "акуратність" Логістичної Регресії зростає, а разом з нею і відрив від Логістичної Регресії + TF-IDF. Починаючи з `TRAIN_SIZE`=5000 ця різниця починає спадати і "акуратність" Логістичної Регресії + TF-IDF починає переганяти "акуратність" просто Логістичної Регресії

Отже, варто використовувати TF-IDF на більших наборах даних

# Embeddings matrix

Досі для доступу до векторів слів ми користувалися бібліотекою `gensim`, яка надавала нам інтерфейс словника (`dict`).

Під капотом, вектори слів зберігаються в одній матриці розмірності $|V| \times d$, де $|V|$ це розмір словника (скільки слів маємо), а $d$ — розмір вектора слова (в цій лабораторній було $d=200$)

In [54]:
glove.vectors.shape

(50000, 200)

В моделях глибинного навчання, як правило, справу мають саме з цією embeddings matrix.

Розглянемо два способи отримати вектор потрібного слова з цієї матриці:

## Vector-matrix multiplication

Перший спосіб, це представити слово з індексом $i$ у вигляді one-hot вектора $o_i$. Тоді ембедінг потрібного слова можна отримати в результаті добутку

$$e_i = \text{E}^\intercal o_i $$

In [55]:
import torch

embeddings_matrix = torch.tensor(glove.vectors)

def embed(token_index: int, embeddings_matrix: torch.tensor) -> torch.tensor:
    vocab_size, embed_dim = embeddings_matrix.shape
    one_hot = torch.zeros(vocab_size)
    one_hot[token_index] = 1
    return one_hot @ embeddings_matrix

assert torch.allclose(
    embed(42, embeddings_matrix),
    torch.tensor(glove.vectors[42]))


In [56]:
lab.checkpoint("The embedding of one-hot multiplication",
               embed(42, embeddings_matrix).sum())

'tensor(5.3482)'

## nn.Embedding

В PyTorch, як і в більшості deep learning frameworks, є спеціальна функція, яка повертає вектор з потрібним номером: `torch.nn.Embedding`. Вона імплементована більш ефективно, ніж спосіб з vector-matrix multiplication, тож в більшості випадків користуватися варто саме `nn.Embedding`.

In [57]:
import torch
from torch import nn

embeddings = nn.Embedding(num_embeddings=50_000, embedding_dim=200, _weight=embeddings_matrix)
indexes = torch.LongTensor([42])
embedded = embeddings(indexes)

assert np.isclose(embedded.sum().item(), glove.vectors[42].sum())

In [58]:
lab.checkpoint("nn.Embeddings", embedded.sum().item())

5.34816837310791

# Готово!

In [59]:
lab.answer("ALL DONE! 😊")

Відповідь правильна ✅
💪



'ALL DONE! 😊'